# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [16]:
# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(10) # 10秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

data = {}
key = ''
date = ''
items = table.find_all('td')
for item in items:
    if 'style' in item.attrs:
        key = item.text
        data[key]={}
    elif 'class' not in item.attrs:
        if item.text.strip() != '':
            if '/' in item.text:
                date = item.text
            else:
                data[key][date] = item.text
# print(data) # 取出所有資料

import pandas as pd
all_data = pd.DataFrame(data)[:-1]
all_data = pd.DataFrame(data)[:-1]
print(all_data[:8]['SO2'])

2018/01    1.80
2018/02    1.90
2018/03    2.20
2018/04    2.30
2018/05    3.10
2018/06    2.70
2018/07    2.20
2018/08    2.40
Name: SO2, dtype: object


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [17]:
print(all_data[:8][['SO2', 'CO']]) # 補上CO

          SO2    CO
2018/01  1.80  0.34
2018/02  1.90  0.44
2018/03  2.20  0.40
2018/04  2.30  0.38
2018/05  3.10  0.34
2018/06  2.70  0.29
2018/07  2.20  0.21
2018/08  2.40  0.30


In [1]:
'''
官方寫法(迴圈開始)
'''
# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(10) # 10秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

d = {}
for tr in table.find_all('tr')[1:]: # 跳過column欄位項目

    for i, td in enumerate(tr.find_all('td')):
        
        if len(tr.find_all('td')) == 5: # 有5個td(欄目) + 第一個月資料
            if i == 0: # 第一個td是監測項目
                obs = td.text
                d.setdefault(obs, {}) # {'項目名稱': {}}
            if i == 2: # 第一個月監測日期
                date = td.text
            if i == 3: # 第一個監測值
                value = td.text
                d[obs][date] = value  # 字典裡面還是字典，進行新增資料
        
        if len(tr.find_all('td')) == 3: # 有3個td為第二個月之後資料
            if i == 0: # 第二個月監測日期
                date = td.text
            if i == 1: # 第二個監測值
                value = td.text
                d[obs][date] = value
print(d)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40', '2018/09': '2.10', '2018/10': '1.70', '2018/11': '1.90', '2018/12': '1.80'}, 'CO': {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30', '2018/09': '0.26', '2018/10': '0.29', '2018/11': '0.30', '2018/12': '0.35'}, 'O3': {'2018/01': '33.40', '2018/02': '32.50', '2018/03': '35', '2018/04': '38.40', '2018/05': '31.60', '2018/06': '29.50', '2018/07': '18.70', '2018/08': '26.40', '2018/09': '29.10', '2018/10': '45.90', '2018/11': '32.40', '2018/12': '30.70'}, 'PM10': {'2018/01': '23', '2018/02': '41', '2018/03': '39', '2018/04': '48', '2018/05': '37', '2018/06': '26', '2018/07': '24', '2018/08': '26', '2018/09': '28', '2018/10': '33', '2018/11': '25', '2018/12': '21'}, 'NOx': {'2018/01': '14.17', '2018/02': '19.43', '2018/03': '18.50',

In [2]:
# 整理成 DataFrame
import pandas as pd

d_SO2 = [{'時間': key, 'SO2': value} for key, value in d['SO2'].items() if key < '2018/09' ]
d_CO = [{'時間': key, 'CO': value} for key, value in d['CO'].items() if key < '2018/09' ]

pd.merge(pd.DataFrame(d_SO2), pd.DataFrame(d_CO))

,時間,SO2,CO
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30


In [3]:
# 整理成 DataFrame
import pandas as pd

d_SO2 = [{'時間': key, 'SO2': value} for key, value in d['SO2'].items() if key < '2018/09' ]
d_CO = [{'時間': key, 'CO': value} for key, value in d['CO'].items() if key < '2018/09' ]

pd.merge(pd.DataFrame(d_SO2), pd.DataFrame(d_CO))

,時間,SO2,CO
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30


In [ ]:
其他寫法:

# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(15) # 15秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

# 創建dictionary
data = {}
for tr in table.find_all('tr')[1:]: # 跳過column欄位項目
    for i, td in enumerate(tr.find_all('td')):
        if len(tr.find_all('td')) == 5:      # 有5個td(欄目) + 第一個月資料
            if i == 0:     # 第一個td是監測項目
                items = td.text
                data.setdefault(items, {})      # {'項目名稱': {}}
            if i == 2:     # 第一個月監測日期
                date = td.text
            if i == 3:     # 第一個監測值
                value = td.text
                data[items][date] = value     # 字典裡面還是字典，進行新增資料
        if len(tr.find_all('td')) == 3:       # 有3個td為第二個月之後資料
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                data[items][date] = value
                
# print(data) # 取出所有data
                
import pandas as pd
all_data = pd.DataFrame(data)[:-1]
all_data = pd.DataFrame(data)[:-1]
print(all_data[:8]['SO2'])